LLMが登場する以前の言語翻訳モデルは、エンコーダとデコーダの2つのサブモジュールを持つディープニューラルネットワークを使うこと。エンコーダの役目は、最初にテキスト全体を読み込んで処理することであり、デコーダの役目は、続いて翻訳されたテキストを生成すること。
Transformer以前は、RNN。前のステップの出力が現在のステップの入力として渡されるニューラルネットワークの一種であり、テキストのような連続的なデータに適している。


self-Attentionとは、シーケンスの表現を計算する時に、入力シーケンス内の各位置が同じシーケンス内の他の全ての1の関連性を考慮できるようにするメカニズムのことです。
このメカニズムでは、それぞれの位置が他の位置にその重要度に応じて「注意を払う」ことができる。

訓練可能な重みを一切持たない、単純化されたSelf-Attentionメカニズムを実装する。

In [3]:
import torch
inputs = torch.tensor(
    [
        [0.43,0.15,0.89],
        [0.55,0.87,0.66],
        [0.57,0.85,0.64],
        [0.22,0.58,0.33],
        [0.77,0.25,0.10],
        [0.05,0.8,0.55]
    ]
)

self-Attentionを実装する最初のステップは、Attentionスコアと呼ばれる中間値wを計算することです。先のinputsテンソルの値を切り捨てており、例えば、0.87は0.8に切り捨てられている。

In [4]:
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i,x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i,query)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [5]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention weights: ",attn_weights_2_tmp)
print("Sum:",attn_weights_2_tmp.sum())

Attention weights:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


In [6]:
#実際の正規化にはソフトマックス関数使うのがベター
def softmax_naive (x):
    return  torch.exp(x) / torch.exp(x).sum(dim = 0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print("Attention weights:", attn_weights_2_naive)
print("Sum :",attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum : tensor(1.)


In [7]:
attn_weights_2 = torch.softmax(attn_scores_2,dim=0)
print("Attention weights:", attn_weights_2)
print("Sum :",attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum : tensor(1.)


In [8]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [9]:
attn_scores = torch.empty(6,6)
for i,x_i in enumerate(inputs):
    for j,x_j in enumerate(inputs):
        attn_scores[i,j] = torch.dot(x_i,x_j)
        
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [10]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [ ]:
attn_weights = torch.softmax(attn_scores,dim=-1)
print(attn_scores)

TypeError: softmax() received an invalid combination of arguments - got (Tensor, dim=float), but expected one of:
 * (Tensor input, int dim, torch.dtype dtype, *, Tensor out)
 * (Tensor input, name dim, *, torch.dtype dtype)
